In [1]:
%matplotlib inline

import os
import torch
import matplotlib.pyplot as plt

from torchvision import transforms, models
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from scipy.optimize import fmin
from skimage import io, transform, img_as_float
from torch.utils.data import Dataset, DataLoader

os.chdir('..') # change working directory to 1 level up

LOCAL=True # SET THIS
GPU=False # SET THIS

In [2]:
nbs = 32 # num examples in batch
ncl = 28 # num classes

In [3]:
bs = (nbs, ncl) # batch array size
# preds = np.random.rand(*bs) # random predictions for batch 
# preds = torch.Tensor(preds)

In [4]:
np.random.RandomState(seed=193952)
true_thresholds = np.random.rand((ncl))

def gen_data(th_true, nbs, ncl):
    data = np.zeros(bs)
    for ix, th in enumerate(th_true):
        col = np.random.normal(loc=th, scale=1, size=(nbs))
        data[:, ix] = col
    return data

In [5]:
preds = gen_data(true_thresholds, nbs, ncl)

In [6]:
# for i in range(ncl):
#     plt.hist(preds[:, i], density=True)
#     plt.vlines(true_thresholds[i], 0, 1, color='r')
#     plt.show()

In [7]:
true_labels = np.zeros(bs)
for i in range(ncl):
    th = true_thresholds[i]
    col = np.zeros(nbs)
    for j in range(nbs):
        col[j] = 1 if np.random.random() > th else 0
    true_labels[:, i] = col

In [8]:
# f1 = lambda params: f1_score(true_labels, preds > params, average='macro')
f1 = lambda params, preds, true_labels: f1_score(true_labels, preds > params, average='macro')

fmin(f1, np.ones(ncl) * .5, (preds, true_labels))

/Users/gmacmillan/anaconda3/envs/k-protein/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Optimization terminated successfully.
         Current function value: 0.361565
         Iterations: 10
         Function evaluations: 271


array([0.5    , 0.5    , 0.5    , 0.5    , 0.5    , 0.5    , 0.5    ,
       0.5    , 0.5    , 0.5    , 0.5    , 0.5    , 0.5    , 0.5    ,
       0.5    , 0.5    , 0.51875, 0.5    , 0.5    , 0.5    , 0.5    ,
       0.5    , 0.5    , 0.50625, 0.5    , 0.5    , 0.5    , 0.5    ])

In [12]:
def f1_loss(pred, labels):
    tp = (pred + labels.data.byte()).eq(2).sum().float()
    fp = (pred - labels.data.byte()).eq(1).sum().float()
    fn = (pred - labels.data.byte()).eq(255).sum().float()
    tn = (pred + labels.data.byte()).eq(0).sum().float()
    try:
        prec = tp / (tp + fp)
    except ZeroDivisionError:
        prec = 0.0
    try:
        rec = tp / (tp + fn)
    except ZeroDivisionError:
        rec = 0.0
        
    f1 = 2 * prec * rec / (prec + rec + 1e-8)
    return 1 - f1.mean()

In [59]:
x = torch.from_numpy(preds).float()
y_true = torch.from_numpy(true_labels).byte()
w = torch.Tensor(np.zeros(ncl)).float()
w = torch.nn.Parameter(w, requires_grad=True)
optimizer = torch.optim.LBFGS([w.requires_grad_()])

for i in range(500):
    def closure():
        optimizer.zero_grad()
        pred = x > w
        loss = f1_loss(pred, y_true)
        loss = Variable(loss, requires_grad = True)
        loss.backward()
        return loss
    optimizer.step(closure)

In [60]:
w

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)